In [13]:
import tensorflow as tf
import numpy as np
import time

import sys
import os

sys.path.append(os.getcwd()+'/models/tutorials/image/cifar10')
print(sys.path)

['/home/yixue/zhanghh/zftech/掌飞-项目集/CoeusAI_AICloud实验资料集/02-TensorFlow实现CNN对CIFAR-10数据分类', '/home/yixue/anaconda3/lib/python37.zip', '/home/yixue/anaconda3/lib/python3.7', '/home/yixue/anaconda3/lib/python3.7/lib-dynload', '', '/home/yixue/anaconda3/lib/python3.7/site-packages', '/home/yixue/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/home/yixue/.ipython', '/home/yixue/zhanghh/zftech/掌飞-项目集/CoeusAI_AICloud实验资料集/02-TensorFlow实现CNN对CIFAR-10数据分类models.tutorials.image.cifar10', '/home/yixue/zhanghh/zftech/掌飞-项目集/CoeusAI_AICloud实验资料集/02-TensorFlow实现CNN对CIFAR-10数据分类models.tutorials.image.cifar10']
/home/yixue/zhanghh/zftech/掌飞-项目集/CoeusAI_AICloud实验资料集/02-TensorFlow实现CNN对CIFAR-10数据分类
['/home/yixue/zhanghh/zftech/掌飞-项目集/CoeusAI_AICloud实验资料集/02-TensorFlow实现CNN对CIFAR-10数据分类', '/home/yixue/anaconda3/lib/python37.zip', '/home/yixue/anaconda3/lib/python3.7', '/home/yixue/anaconda3/lib/python3.7/lib-dynload', '', '/home/yixue/anaconda3/lib/python3.7/site-packages', '/home/yixue/anac

In [18]:
from models.tutorials.image.cifar10 import cifar10, cifar10_input
cifar10.maybe_download_and_extract()

UnrecognizedFlagError: Unknown command line flag 'f'

In [24]:
import sys
import os

sys.path.append(os.getcwd()+'/models/tutorials/image/cifar10')

from models.tutorials.image.cifar10 import cifar10, cifar10_input

import tensorflow as tf
import numpy as np
import time


def variable_with_weight_loss(shape, stddev, wl):
    '''
    使用tf.truncated_normal截断的正态分布来初始化权重,这里给weight加一个L2的loss.
    我们使用wl控制L2 loss的大小,使用tf.nn.l2_loss计算weight的L2 loss.
    再使用tf.multiply让L2 loss乘wl,得到最后的weight loss,最后将weight loss添加到一个collection.便于后期优化
    :param shape:
    :param stddev:
    :param wl:
    :return:
    '''
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), wl, name='weight_loss')
        tf.add_to_collection('losses',weight_loss)
    return var


def loss(logits, labels):
    '''
    使用tf.nn.sparse_softmax_cross_entropy_with_logits将softmax和cross_entropy_loss计算合在一起
    并计算cross_entropy的均值添加到losses集合.以便于后面输出所有losses
    :param logits:
    :param labels:
    :return:
    '''
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                        labels=labels, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses',cross_entropy_mean)

    return tf.add_n(tf.get_collection('losses'), name='total_loss')


def train():
    max_steps = 30000
    batch_size = 128
    data_dir = './data/cifar-10-batches-bin'

    '''
    使用cifar10_input类中的disorted_inputs函数产生训练数据,产生的数据是已经封装好的Tensor,每次会产生batch_size个
    这个函数已经对图片数据做了增强操作(随机水平翻转/剪切/随机对比度等)
    同时因为对图像处理需要耗费大量计算资源,该函数使用了16个独立的线程来加速任务,
    函数内部会产生线程池,使用会通过TensorFlow queue进行调度
    '''
    images_train,labels_train = cifar10_input.distorted_inputs(data_dir=data_dir, batch_size=batch_size)
    images_test,labels_test = cifar10_input.inputs(eval_data=True, data_dir=data_dir, batch_size=batch_size)

    image_holder = tf.placeholder(tf.float32, [batch_size, 24 ,24, 3])
    label_holder = tf.placeholder(tf.int32, [batch_size])

    # 第一层 卷积-->池化-->lrn
    # 不带L2正则项(wl=0)的64个5x5x3的滤波器,
    # 使用lrn是从局部多个卷积核的响应中挑选比较大的反馈变得相对最大,并抑制其他反馈小的,增加模型泛化能力
    weight1 = variable_with_weight_loss(shape=[5, 5, 3, 64], stddev=5e-2, wl=0.0)
    kernel1 = tf.nn.conv2d(image_holder, weight1, strides=[1, 1, 1, 1], padding='SAME')
    bias1 = tf.Variable(tf.constant(0.0, shape=[64])) #bias直接初始化为0
    conv1 = tf.nn.relu(tf.nn.bias_add(kernel1, bias1))
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)

    # 第二层  卷积-->lrn-->池化
    weight2 = variable_with_weight_loss(shape=[5, 5, 64, 64], stddev=5e-2, wl=0.0)
    kernel2 = tf.nn.conv2d(norm1, weight2, strides=[1, 1, 1, 1], padding='SAME')
    bias2 = tf.Variable(tf.constant(0.1,shape=[64]))
    conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0,alpha=0.001/9.0,beta=0.75)
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1],padding='SAME')


    #第三层 使用全连接层 reshape后获取长度并创建FC1层的权重(带L2正则化)
    reshape = tf.reshape(pool2, [batch_size, -1])
    dim = reshape.get_shape()[1].value
    weight3 = variable_with_weight_loss(shape=[dim, 384], stddev=0.04, wl=0.004)
    bias3 = tf.Variable(tf.constant(0.1,shape=[384]))
    local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

    #第四层 FC2层  节点数减半  依旧带L2正则
    weight4 = variable_with_weight_loss(shape=[384, 192], stddev=0.04, wl=0.004)
    bias4 = tf.Variable(tf.constant(0.1, shape=[192]))
    local4 = tf.nn.relu(tf.matmul(local3,weight4) + bias4)

    #最后一层 这层的weight设为正态分布标准差设为上一个FC层的节点数的倒数
    #这里我们不计算softmax,把softmax放到后面计算
    weight5 = variable_with_weight_loss(shape=[192,10], stddev=1/192.0, wl=0.0)
    bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
    logits = tf.add(tf.matmul(local4, weight5),bias5)

    #损失函数为两个带L2正则的FC层和最后的转换层
    #优化器依旧是AdamOptimizer,学习率是1e-3
    losses = loss(logits, label_holder)
    train_op = tf.train.AdamOptimizer(1e-3).minimize(losses)

    #in_top_k函数求出输出结果中top k的准确率,这里选择输出top1
    top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

    #创建默认session,初始化变量
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()

    #启动图片增强线程队列
    tf.train.start_queue_runners()

    #训练
    for step in range(max_steps):
        start_time = time.time()
        image_batch,label_batch = sess.run([images_train, labels_train])
        _,loss_value = sess.run([train_op, losses], feed_dict={image_holder: image_batch,
                                                            label_holder: label_batch})
        duration = time.time() - start_time

        if step % 10 == 0:
            examples_per_sec = batch_size / duration
            sec_per_batch = float(duration)

            format_str = ('step %d,loss=%.2f (%.1f examples/sec; %.3f sec/batch)')
            print(format_str % (step, loss_value, examples_per_sec, sec_per_batch))

    #评估模型的准确率,测试集一共有10000个样本
    #我们先计算大概要多少个batch能测试完所有样本
    num_examples = 10000
    import math
    num_iter = int(math.ceil(num_examples / batch_size))
    true_count = 0
    total_sample_count = num_iter * batch_size  #除去不够一个batch的
    step = 0
    while step < num_iter:
        image_batch, label_batch = sess.run([images_test, labels_test])
        predictions = sess.run([top_k_op], feed_dict={image_holder: image_batch,
                                                      label_holder: label_batch})
        true_count += np.sum(predictions)  #利用top_k_op计算输出结果
        step += 1

    precision = true_count / total_sample_count
    print('precision @ 1=%.3f' % precision)  #这里如果输出为0.00 是因为整数/整数  记得要导入float除法


# if __name__ == '__main__':
#     cifar10.maybe_download_and_extract()
#     train()
print('hello')
# tf.app.flags.DEFINE_string('f','','kernel') 
cifar10.maybe_download_and_extract()

train()

hello
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.F

step 630,loss=1.50 (2920.8 examples/sec; 0.044 sec/batch)
step 640,loss=1.49 (2714.1 examples/sec; 0.047 sec/batch)
step 650,loss=1.53 (2729.5 examples/sec; 0.047 sec/batch)
step 660,loss=1.47 (3026.5 examples/sec; 0.042 sec/batch)
step 670,loss=1.46 (2757.9 examples/sec; 0.046 sec/batch)
step 680,loss=1.38 (2863.2 examples/sec; 0.045 sec/batch)
step 690,loss=1.33 (2981.5 examples/sec; 0.043 sec/batch)
step 700,loss=1.47 (2811.4 examples/sec; 0.046 sec/batch)
step 710,loss=1.39 (2796.7 examples/sec; 0.046 sec/batch)
step 720,loss=1.36 (3042.2 examples/sec; 0.042 sec/batch)
step 730,loss=1.27 (2868.1 examples/sec; 0.045 sec/batch)
step 740,loss=1.15 (3097.5 examples/sec; 0.041 sec/batch)
step 750,loss=1.44 (3114.6 examples/sec; 0.041 sec/batch)
step 760,loss=1.24 (2837.9 examples/sec; 0.045 sec/batch)
step 770,loss=1.32 (2840.0 examples/sec; 0.045 sec/batch)
step 780,loss=1.41 (2837.6 examples/sec; 0.045 sec/batch)
step 790,loss=1.37 (3046.4 examples/sec; 0.042 sec/batch)
step 800,loss=

step 2030,loss=1.09 (3033.2 examples/sec; 0.042 sec/batch)
step 2040,loss=1.08 (2809.0 examples/sec; 0.046 sec/batch)
step 2050,loss=1.28 (3006.8 examples/sec; 0.043 sec/batch)
step 2060,loss=1.11 (2936.2 examples/sec; 0.044 sec/batch)
step 2070,loss=1.08 (2873.0 examples/sec; 0.045 sec/batch)
step 2080,loss=1.16 (2997.3 examples/sec; 0.043 sec/batch)
step 2090,loss=1.04 (2954.3 examples/sec; 0.043 sec/batch)
step 2100,loss=1.35 (2872.6 examples/sec; 0.045 sec/batch)
step 2110,loss=1.24 (2917.2 examples/sec; 0.044 sec/batch)
step 2120,loss=1.08 (2910.4 examples/sec; 0.044 sec/batch)
step 2130,loss=0.99 (2991.8 examples/sec; 0.043 sec/batch)
step 2140,loss=1.22 (2898.5 examples/sec; 0.044 sec/batch)
step 2150,loss=1.34 (2921.2 examples/sec; 0.044 sec/batch)
step 2160,loss=1.03 (2999.4 examples/sec; 0.043 sec/batch)
step 2170,loss=1.05 (3011.5 examples/sec; 0.043 sec/batch)
step 2180,loss=1.25 (2851.8 examples/sec; 0.045 sec/batch)
step 2190,loss=1.04 (3009.8 examples/sec; 0.043 sec/batc

step 3420,loss=0.91 (2985.8 examples/sec; 0.043 sec/batch)
step 3430,loss=1.13 (3060.9 examples/sec; 0.042 sec/batch)
step 3440,loss=0.97 (2827.7 examples/sec; 0.045 sec/batch)
step 3450,loss=0.93 (2863.6 examples/sec; 0.045 sec/batch)
step 3460,loss=0.82 (2859.8 examples/sec; 0.045 sec/batch)
step 3470,loss=0.91 (3168.6 examples/sec; 0.040 sec/batch)
step 3480,loss=0.87 (2901.9 examples/sec; 0.044 sec/batch)
step 3490,loss=0.95 (3043.6 examples/sec; 0.042 sec/batch)
step 3500,loss=0.92 (2879.8 examples/sec; 0.044 sec/batch)
step 3510,loss=1.05 (3157.9 examples/sec; 0.041 sec/batch)
step 3520,loss=1.12 (3187.6 examples/sec; 0.040 sec/batch)
step 3530,loss=0.97 (2860.2 examples/sec; 0.045 sec/batch)
step 3540,loss=0.96 (2874.9 examples/sec; 0.045 sec/batch)
step 3550,loss=0.97 (2956.1 examples/sec; 0.043 sec/batch)
step 3560,loss=1.15 (2964.1 examples/sec; 0.043 sec/batch)
step 3570,loss=1.02 (2617.7 examples/sec; 0.049 sec/batch)
step 3580,loss=0.90 (2705.8 examples/sec; 0.047 sec/batc

step 4810,loss=1.02 (2730.8 examples/sec; 0.047 sec/batch)
step 4820,loss=0.83 (3330.5 examples/sec; 0.038 sec/batch)
step 4830,loss=1.13 (2908.4 examples/sec; 0.044 sec/batch)
step 4840,loss=1.01 (2722.8 examples/sec; 0.047 sec/batch)
step 4850,loss=1.07 (2960.0 examples/sec; 0.043 sec/batch)
step 4860,loss=1.09 (2835.8 examples/sec; 0.045 sec/batch)
step 4870,loss=1.12 (3022.8 examples/sec; 0.042 sec/batch)
step 4880,loss=0.88 (2986.0 examples/sec; 0.043 sec/batch)
step 4890,loss=0.92 (2927.8 examples/sec; 0.044 sec/batch)
step 4900,loss=0.92 (2867.9 examples/sec; 0.045 sec/batch)
step 4910,loss=0.93 (2951.2 examples/sec; 0.043 sec/batch)
step 4920,loss=0.84 (3207.7 examples/sec; 0.040 sec/batch)
step 4930,loss=1.00 (2815.0 examples/sec; 0.045 sec/batch)
step 4940,loss=0.85 (3026.7 examples/sec; 0.042 sec/batch)
step 4950,loss=1.19 (3011.7 examples/sec; 0.043 sec/batch)
step 4960,loss=1.05 (3034.5 examples/sec; 0.042 sec/batch)
step 4970,loss=0.89 (2777.0 examples/sec; 0.046 sec/batc

...

step 24070,loss=0.83 (2793.6 examples/sec; 0.046 sec/batch)
step 24080,loss=0.86 (2909.3 examples/sec; 0.044 sec/batch)
step 24090,loss=0.78 (2928.7 examples/sec; 0.044 sec/batch)
step 24100,loss=0.54 (2821.2 examples/sec; 0.045 sec/batch)
step 24110,loss=0.68 (2945.1 examples/sec; 0.043 sec/batch)
step 24120,loss=0.72 (2815.1 examples/sec; 0.045 sec/batch)
step 24130,loss=0.53 (2773.0 examples/sec; 0.046 sec/batch)
step 24140,loss=0.72 (2930.7 examples/sec; 0.044 sec/batch)
step 24150,loss=0.56 (2941.3 examples/sec; 0.044 sec/batch)
step 24160,loss=0.77 (2833.6 examples/sec; 0.045 sec/batch)
step 24170,loss=0.73 (2824.2 examples/sec; 0.045 sec/batch)
step 24180,loss=0.75 (3033.7 examples/sec; 0.042 sec/batch)
step 24190,loss=0.74 (2910.8 examples/sec; 0.044 sec/batch)
step 24200,loss=0.67 (3059.9 examples/sec; 0.042 sec/batch)
step 24210,loss=0.97 (2787.8 examples/sec; 0.046 sec/batch)
step 24220,loss=0.66 (2710.3 examples/sec; 0.047 sec/batch)
step 24230,loss=0.69 (3209.0 examples/se

step 25440,loss=0.67 (2940.8 examples/sec; 0.044 sec/batch)
step 25450,loss=0.69 (3119.3 examples/sec; 0.041 sec/batch)
step 25460,loss=0.54 (2832.4 examples/sec; 0.045 sec/batch)
step 25470,loss=0.80 (2906.9 examples/sec; 0.044 sec/batch)
step 25480,loss=0.71 (2822.2 examples/sec; 0.045 sec/batch)
step 25490,loss=0.75 (2874.2 examples/sec; 0.045 sec/batch)
step 25500,loss=0.82 (2987.4 examples/sec; 0.043 sec/batch)
step 25510,loss=0.75 (2825.3 examples/sec; 0.045 sec/batch)
step 25520,loss=0.64 (2954.9 examples/sec; 0.043 sec/batch)
step 25530,loss=0.54 (2926.7 examples/sec; 0.044 sec/batch)
step 25540,loss=0.82 (2982.5 examples/sec; 0.043 sec/batch)
step 25550,loss=0.70 (2944.5 examples/sec; 0.043 sec/batch)
step 25560,loss=0.69 (3070.7 examples/sec; 0.042 sec/batch)
step 25570,loss=0.76 (2819.8 examples/sec; 0.045 sec/batch)
step 25580,loss=0.77 (2664.4 examples/sec; 0.048 sec/batch)
step 25590,loss=0.87 (2817.4 examples/sec; 0.045 sec/batch)
step 25600,loss=0.62 (2967.4 examples/se

step 26810,loss=0.66 (2882.4 examples/sec; 0.044 sec/batch)
step 26820,loss=0.66 (2780.9 examples/sec; 0.046 sec/batch)
step 26830,loss=0.76 (2861.8 examples/sec; 0.045 sec/batch)
step 26840,loss=0.67 (2981.5 examples/sec; 0.043 sec/batch)
step 26850,loss=0.84 (2902.0 examples/sec; 0.044 sec/batch)
step 26860,loss=0.70 (2895.6 examples/sec; 0.044 sec/batch)
step 26870,loss=0.79 (2852.5 examples/sec; 0.045 sec/batch)
step 26880,loss=0.76 (2967.2 examples/sec; 0.043 sec/batch)
step 26890,loss=0.76 (3003.4 examples/sec; 0.043 sec/batch)
step 26900,loss=0.69 (2874.2 examples/sec; 0.045 sec/batch)
step 26910,loss=0.70 (3065.0 examples/sec; 0.042 sec/batch)
step 26920,loss=0.83 (2988.7 examples/sec; 0.043 sec/batch)
step 26930,loss=0.75 (3075.5 examples/sec; 0.042 sec/batch)
step 26940,loss=0.76 (3024.1 examples/sec; 0.042 sec/batch)
step 26950,loss=0.65 (2976.3 examples/sec; 0.043 sec/batch)
step 26960,loss=0.59 (3000.2 examples/sec; 0.043 sec/batch)
step 26970,loss=0.68 (3111.9 examples/se

step 28180,loss=0.60 (2925.8 examples/sec; 0.044 sec/batch)
step 28190,loss=0.68 (3079.9 examples/sec; 0.042 sec/batch)
step 28200,loss=0.75 (2871.5 examples/sec; 0.045 sec/batch)
step 28210,loss=0.59 (2913.6 examples/sec; 0.044 sec/batch)
step 28220,loss=0.76 (2931.6 examples/sec; 0.044 sec/batch)
step 28230,loss=0.65 (2906.9 examples/sec; 0.044 sec/batch)
step 28240,loss=0.77 (3041.3 examples/sec; 0.042 sec/batch)
step 28250,loss=0.75 (2959.1 examples/sec; 0.043 sec/batch)
step 28260,loss=0.77 (3015.3 examples/sec; 0.042 sec/batch)
step 28270,loss=0.81 (2880.7 examples/sec; 0.044 sec/batch)
step 28280,loss=0.97 (2857.9 examples/sec; 0.045 sec/batch)
step 28290,loss=0.79 (2967.9 examples/sec; 0.043 sec/batch)
step 28300,loss=0.61 (2841.2 examples/sec; 0.045 sec/batch)
step 28310,loss=0.67 (3223.8 examples/sec; 0.040 sec/batch)
step 28320,loss=0.61 (3310.4 examples/sec; 0.039 sec/batch)
step 28330,loss=0.76 (2890.4 examples/sec; 0.044 sec/batch)
step 28340,loss=0.80 (3001.7 examples/se

step 29550,loss=0.99 (2857.9 examples/sec; 0.045 sec/batch)
step 29560,loss=0.69 (2855.5 examples/sec; 0.045 sec/batch)
step 29570,loss=0.78 (3023.0 examples/sec; 0.042 sec/batch)
step 29580,loss=0.82 (3077.9 examples/sec; 0.042 sec/batch)
step 29590,loss=0.78 (2747.2 examples/sec; 0.047 sec/batch)
step 29600,loss=0.77 (2811.7 examples/sec; 0.046 sec/batch)
step 29610,loss=0.69 (3197.1 examples/sec; 0.040 sec/batch)
step 29620,loss=0.60 (3229.4 examples/sec; 0.040 sec/batch)
step 29630,loss=0.69 (3237.6 examples/sec; 0.040 sec/batch)
step 29640,loss=0.67 (2763.1 examples/sec; 0.046 sec/batch)
step 29650,loss=0.64 (2921.7 examples/sec; 0.044 sec/batch)
step 29660,loss=0.65 (2948.5 examples/sec; 0.043 sec/batch)
step 29670,loss=0.65 (2736.2 examples/sec; 0.047 sec/batch)
step 29680,loss=0.56 (2989.5 examples/sec; 0.043 sec/batch)
step 29690,loss=0.63 (2709.9 examples/sec; 0.047 sec/batch)
step 29700,loss=0.61 (2954.3 examples/sec; 0.043 sec/batch)
step 29710,loss=0.67 (2968.2 examples/se